# Sandbox
Notebook to test code before real implementation

In [1]:
################ ONLY PYTHON 2 #########################
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations, chain, product
import argparse
import time
import os
import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
import pandas as pd
import psutil
import pickle
from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import os
import psutil
import pickle
from pybdm import BDM
from pybdm.utils import decompose_dataset
from pybdm.partitions import PartitionIgnore
from joblib import Parallel, delayed
from joblib import parallel_backend
from data.algorithms import PerturbationExperiment, NodePerturbationExperiment
import math
import datetime as dt
from itertools import product

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/juan/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
DF = pd.read_csv('../Thesis_datasets/LINCS/LINCS_Gene_Experssion_signatures_CD.csv')

In [4]:
DF.columns

Index(['pert_id', '200814_at', '222103_at', '201453_x_at', '204131_s_at',
       '200059_s_at', '205067_at', '213702_x_at', '214435_x_at', '201334_s_at',
       ...
       '204662_at', '203047_at', '203343_at', '217995_at', '218450_at',
       '212536_at', '218529_at', '211071_s_at', '203341_at', '205379_at'],
      dtype='object', length=979)

In [5]:
len(DF.columns)

979

In [6]:
len(DF.index)

20339

In [8]:
DF.head(5)

,pert_id,200814_at,222103_at,201453_x_at,204131_s_at,200059_s_at,205067_at,213702_x_at,214435_x_at,201334_s_at,...,204662_at,203047_at,203343_at,217995_at,218450_at,212536_at,218529_at,211071_s_at,203341_at,205379_at
0,BRD-A00100033,0.0193,0.0260,-0.0136,0.0283,-0.0279,0.0246,-0.0713,0.0358,-0.0093,...,-0.0257,0.0215,0.0254,-0.0361,-0.0169,-0.0080,0.0073,-0.0984,-0.0145,-0.0149
1,BRD-A00150179,-0.0197,-0.0058,-0.0243,0.0088,-0.0129,-0.0070,0.0621,-0.0018,-0.0053,...,-0.0098,0.0089,-0.0003,0.0064,0.0114,0.0352,0.0309,-0.0212,-0.0115,-0.0257
2,BRD-A00267231,-0.0370,0.0010,0.0115,0.0271,0.0188,-0.0106,-0.0023,0.0031,0.0472,...,-0.0096,-0.0440,0.0119,0.0206,0.0097,-0.0215,-0.0194,0.1062,-0.0114,-0.0262
3,BRD-A00420644,0.0109,-0.0258,0.0221,-0.0354,0.0222,0.0198,0.0334,-0.0218,-0.0048,...,0.0228,0.0140,0.0564,0.0508,0.0083,-0.0135,0.0336,0.0184,0.0256,0.0150
4,BRD-A00474148,-0.0033,0.0042,-0.0131,0.0130,0.0113,0.0560,-0.0398,0.0275,-0.0223,...,0.0142,0.0075,0.0591,0.0514,0.0113,-0.1378,0.0193,0.0462,-0.0252,0.0161
